In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")
import model.sdes as sdes
import model.generate as generate
import model.cifar_unet as cifar_unet
import model.util as model_util
from plot.plot import plot_mnist_digits
from analysis.fid import compute_fid
import torch
import torchvision
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import os

In [ ]:
# Define device
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

In [ ]:
classes = list(range(100))
branch_defs = [((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.6166166166166166, 1), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.5985985985985985, 0.6166166166166166), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.5465465465465466, 0.5985985985985985), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.5415415415415415, 0.5465465465465466), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.5135135135135135, 0.5415415415415415), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.5045045045045045, 0.5135135135135135), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99), 0.5025025025025025, 0.5045045045045045), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99), 0.4824824824824825, 0.5025025025025025), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99), 0.47347347347347346, 0.4824824824824825), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99), 0.45245245245245247, 0.47347347347347346), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99), 0.44344344344344344, 0.45245245245245247), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99), 0.4094094094094094, 0.44344344344344344), ((0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99), 0.3763763763763764, 0.4094094094094094), ((0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99), 0.36736736736736736, 0.3763763763763764), ((0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99), 0.36036036036036034, 0.36736736736736736), ((0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99), 0.2092092092092092, 0.36036036036036034), ((0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 22, 23, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99), 0.15, 0.2092092092092092), ((20, 24), 0.15, 0.5465465465465466), ((86,), 0, 0.6166166166166166), ((73,), 0, 0.5985985985985985), ((69,), 0, 0.5415415415415415), ((62,), 0, 0.5135135135135135), ((9,), 0, 0.5045045045045045), ((95,), 0, 0.5025025025025025), ((30,), 0, 0.4824824824824825), ((17,), 0, 0.47347347347347346), ((67,), 0, 0.45245245245245247), ((92,), 0, 0.44344344344344344), ((41,), 0, 0.4094094094094094), ((7,), 0, 0.3763763763763764), ((33,), 0, 0.36736736736736736), ((54,), 0, 0.36036036036036034), ((21,), 0, 0.2092092092092092), ((36,), 0, 0.15), ((60,), 0, 0.15), ((71,), 0, 0.15), ((61,), 0, 0.15), ((90,), 0, 0.15), ((82,), 0, 0.15), ((85,), 0, 0.15), ((25,), 0, 0.15), ((23,), 0, 0.15), ((24,), 0, 0.15), ((20,), 0, 0.15), ((27,), 0, 0.15), ((28,), 0, 0.15), ((89,), 0, 0.15), ((79,), 0, 0.15), ((94,), 0, 0.15), ((68,), 0, 0.15), ((26,), 0, 0.15), ((31,), 0, 0.15), ((63,), 0, 0.15), ((47,), 0, 0.15), ((4,), 0, 0.15), ((98,), 0, 0.15), ((51,), 0, 0.15), ((1,), 0, 0.15), ((59,), 0, 0.15), ((52,), 0, 0.15), ((96,), 0, 0.15), ((50,), 0, 0.15), ((8,), 0, 0.15), ((58,), 0, 0.15), ((55,), 0, 0.15), ((91,), 0, 0.15), ((93,), 0, 0.15), ((49,), 0, 0.15), ((12,), 0, 0.15), ((66,), 0, 0.15), ((2,), 0, 0.15), ((70,), 0, 0.15), ((83,), 0, 0.15), ((6,), 0, 0.15), ((53,), 0, 0.15), ((16,), 0, 0.15), ((19,), 0, 0.15), ((5,), 0, 0.15), ((32,), 0, 0.15), ((0,), 0, 0.15), ((43,), 0, 0.15), ((13,), 0, 0.15), ((18,), 0, 0.15), ((29,), 0, 0.15), ((48,), 0, 0.15), ((87,), 0, 0.15), ((88,), 0, 0.15), ((10,), 0, 0.15), ((39,), 0, 0.15), ((22,), 0, 0.15), ((56,), 0, 0.15), ((57,), 0, 0.15), ((40,), 0, 0.15), ((45,), 0, 0.15), ((42,), 0, 0.15), ((64,), 0, 0.15), ((75,), 0, 0.15), ((65,), 0, 0.15), ((46,), 0, 0.15), ((72,), 0, 0.15), ((74,), 0, 0.15), ((77,), 0, 0.15), ((78,), 0, 0.15), ((80,), 0, 0.15), ((84,), 0, 0.15), ((97,), 0, 0.15), ((35,), 0, 0.15), ((99,), 0, 0.15), ((76,), 0, 0.15), ((37,), 0, 0.15), ((81,), 0, 0.15), ((3,), 0, 0.15), ((11,), 0, 0.15), ((14,), 0, 0.15), ((15,), 0, 0.15), ((34,), 0, 0.15), ((38,), 0, 0.15), ((44,), 0, 0.15)]
# branch_defs = [(tuple(classes), 0, 1)]

# classes = [0, 53]
# branch_defs = [
#     ((0, 53), 0.5, 1),
#     ((0,), 0, 0.5),
#     ((53,), 0, 0.5)
# ]

In [ ]:
class CIFARBranchDataset(torch.utils.data.IterableDataset):

    def __init__(self, classes, branch_defs, batch_size, num_batches):
        self.class_images = {}

        cifar_dataset = torchvision.datasets.CIFAR100(
            "/gstore/scratch/u/tsenga5/datasets/CIFAR-100", train=True
        )
        cifar_dataset.targets = np.array(cifar_dataset.targets)
        for c in classes:
            inds = cifar_dataset.targets == c
            data = cifar_dataset.data[inds]
            self.class_images[c] = (np.transpose(data, (0, 3, 1, 2)) / 256 * 2) - 1

        self.branch_defs = branch_defs
        self.branch_inds = np.tile(
            np.arange(len(branch_defs)), int(np.ceil(num_batches / len(branch_defs)))
        )[:num_batches]
        self.branch_inds = np.random.permutation(self.branch_inds)
        self.batch_size = batch_size

    def get_batch(self, index):
        classes, time_start, time_end = self.branch_defs[self.branch_inds[index]]
        data = []
        for _ in range(self.batch_size):
            images = self.class_images[np.random.choice(classes)]
            data.append(images[np.random.choice(len(images))])
        data = np.stack(data)
        times = (np.random.rand(self.batch_size) * (time_end - time_start)) + time_start
        branch_inds = np.full(self.batch_size, self.branch_inds[index])
        return torch.tensor(data), torch.tensor(times), torch.tensor(branch_inds)

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        num_batches = len(self)
        if worker_info is None:
            # In single-processing mode
            start, end = 0, num_batches
        else:
            worker_id = worker_info.id
            num_workers = worker_info.num_workers
            shard_size = int(np.ceil(num_batches / num_workers))
            start = shard_size * worker_id
            end = min(start + shard_size, num_batches)
        return (self.get_batch(i) for i in range(start, end))

    def __len__(self):
        return len(self.branch_inds)

    def on_epoch_start(self):
        self.branch_inds = np.random.permutation(self.branch_inds)
        
dataset = CIFARBranchDataset(classes, branch_defs, 16, 300)

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=None, num_workers=2, collate_fn=lambda x: x
)
input_shape = next(iter(data_loader))[0].shape[1:]

In [ ]:
# dataset = torchvision.datasets.CIFAR100(
#     "/gstore/scratch/u/tsenga5/datasets/CIFAR-100", train=True, transform=(lambda img: (np.transpose(np.asarray(img), (2, 0, 1)) / 256 * 2) - 1)
# )

# # Limit classes
# dataset.targets = np.array(dataset.targets)
# inds = np.isin(dataset.targets, classes)
# dataset.data = dataset.data[inds]
# dataset.targets = dataset.targets[inds]

# data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=2)
# sample_x, sample_y = next(iter(data_loader))
# input_shape = sample_x.shape[1:]

In [ ]:
# TODO: this is currently rather inefficient; a decision-tree-style structure
# would be better

def class_time_to_branch(c, t):
    """
    Given a class and a time (both scalars), return the
    corresponding branch index.
    """
    for i, branch_def in enumerate(branch_defs):
        if c in branch_def[0] and t >= branch_def[1] and t <= branch_def[2]:
            return i
    raise ValueError("Undefined class and time")
        
def class_time_to_branch_tensor(c, t):
    """
    Given tensors of classes and a times, return the
    corresponding branch indices as a tensor.
    """
    return torch.tensor([
        class_time_to_branch(c_i, t_i) for c_i, t_i in zip(c, t)
    ], device=DEVICE)

In [ ]:
# Create the SDE and model
sde = sdes.VariancePreservingSDE(0.1, 20, input_shape)

t_limit = 1
model = cifar_unet.MultitaskUNet(
    len(branch_defs), t_limit=t_limit,
    shared_layers=[True, True, True, True, True, True, True, False, False]
).to(DEVICE)

os.environ["MODEL_DIR"] = "/gstore/scratch/u/tsenga5/branched_diffusion/models/trained_models/cifar_continuous_branched_100classes_warm"

import model.train_continuous_model as train_continuous_model  # Import this AFTER setting environment

In [ ]:
def plot_cifar(
    images, grid_size=(10, 5), scale=1, clip=False, title=None
):
    """
    Plots CIFAR objects. No normalization will be done.
    Arguments:
        `images`: a B x 3 x 28 x 28 NumPy array of numbers between
            0 and 1
        `grid_size`: a pair of integers denoting the number of images
            to plot horizontally and vertically (in that order); if
            more digits are provided than spaces in the grid, leftover
            digits will not be plotted; if fewer images are provided
            than spaces in the grid, there will be at most one
            unfinished row
        `scale`: amount to scale figure size by
        `clip`: if True, clip values to between 0 and 1
        `title`: if given, title for the plot
    """
    images = np.transpose(images, (0, 2, 3, 1))
    if clip:
        images = np.clip(images, 0, 1)

    width, height = grid_size
    num_images = len(images)
    height = min(height, num_images // width)

    figsize = (width * scale, (height * scale) + 0.5)

    fig, ax = plt.subplots(
        ncols=width, nrows=height,
        figsize=figsize, gridspec_kw={"wspace": 0, "hspace": 0}
    )
    if height == 1:
        ax = [ax]
    if width == 1:
        ax = [[a] for a in ax]

    for j in range(height):
        for i in range(width):
            index = i + (width * j)
            ax[j][i].imshow(images[index], cmap="gray", aspect="auto", interpolation=None)
            ax[j][i].axis("off")
    if title:
        ax[0][0].set_title(title)
    plt.subplots_adjust(bottom=0.25)
    return fig

In [ ]:
# Show some images after running the SDE forward for different times
time_steps = 4

x0, _ = next(iter(data_loader))
x0 = x0.cpu().numpy()

plot_cifar((x0 + 1) / 2, grid_size=(8, 1), clip=True, title="t = 0")
x0 = torch.tensor(x0).to(DEVICE)
for t in np.linspace(0.01, t_limit, time_steps):
    xt, score = sde.forward(x0, torch.full(x0.shape[:1], t).to(DEVICE))
    plot_cifar((xt.cpu().numpy() + 1) / 2, grid_size=(8, 1), clip=True, title=("t = %.2f" % t))
    
# Show the transformation of the distribution of data to the prior distribution
time_steps = 30

all_t = np.linspace(0, t_limit, time_steps)
all_xt = np.empty((len(all_t),) + x0.shape)
for t_i, t in enumerate(all_t):
    xt, _ = sde.forward(x0, torch.ones(len(x0)).to(DEVICE) * t)
    all_xt[t_i] = xt.cpu().numpy()

fig, ax = plt.subplots(figsize=(20, 8))
cmap = plt.get_cmap("magma")
for t_i in range(len(all_t)):
    ax.hist(np.ravel(all_xt[t_i]), bins=60, histtype="step", color=cmap(t_i / len(all_t)), alpha=0.5, density=True)
prior = sde.sample_prior(len(x0), torch.ones(len(x0)).to(DEVICE) * t).cpu().numpy()
ax.hist(np.ravel(prior), bins=60, histtype="step", color="blue", linewidth=2, density=True, label="Sampled prior")
ax.set_xlabel("x")
ax.set_ylabel("p(x)")
ax.set_title("Evolution of p(x) over forward SDE")
ax.set_ylim((0, 3))
plt.legend()
plt.show()

In [ ]:
model_init = model_util.load_model(
    cifar_unet.MultitaskUNet,
    os.path.join(
        "/gstore/scratch/u/tsenga5/branched_diffusion/models/trained_models/",
        "cifar_continuous_nobranch/1/last_ckpt.pth"
    )
).to(DEVICE)

In [ ]:
# Copy over weights
for module_name, module in model_init.named_children():
    if module_name in ("init_conv", "time_mlp"):
        model.get_submodule(module_name).load_state_dict(module.state_dict())
    elif module_name in ("mid", "final_res_blocks", "final_convs"):
        target_module = model.get_submodule(module_name)
        for i in range(len(target_module)):
            target_module[i].load_state_dict(module[0].state_dict())
    elif module_name in ("ups", "downs"):
        for layer_i in range(len(module)):
            target_module = model.get_submodule(module_name)[layer_i]
            for i in range(len(target_module)):
                target_module[i].load_state_dict(module[layer_i][0].state_dict())
    else:
        raise ValueError()

In [ ]:
train_continuous_model.train_ex.run(
    "train_branched_model",
    config_updates={
        "model": model,
        "sde": sde,
        "data_loader": data_loader,
        "class_time_to_branch_index": class_time_to_branch_tensor,
        "num_epochs": 120,
        "learning_rate": 0.00001,
        "t_limit": t_limit,
        "loss_weighting_type": "empirical_norm",
        "data_loader_returns_t_and_b": True
    }
)

In [ ]:
gen_samples = {}
for class_to_sample in classes:
    print("Sampling class: %s" % class_to_sample)
    samples = generate.generate_continuous_branched_samples(
        model, sde, class_to_sample, class_time_to_branch_tensor,
        sampler="pc", t_limit=t_limit
    ).cpu().numpy()
    gen_samples[class_to_sample] = samples
    plot_cifar((samples + 1) / 2, grid_size=(10, 1), clip=True, title=None)
    plt.show()

In [ ]:
true_samples = {}
for class_to_sample in classes:
    print("Sampling class: %s" % class_to_sample)
    inds = np.where(dataset.targets == class_to_sample)[0]
    sample_inds = np.random.choice(inds, size=200, replace=False)
    samples = np.transpose((dataset.data[sample_inds] / 256 * 2) - 1, (0, 3, 1, 2))
    true_samples[class_to_sample] = samples
    plot_cifar((samples + 1) / 2, grid_size=(10, 1), title=None)

In [ ]:
# Compute FID scores
fid_scores = {}
for class_to_sample in classes:
    print("FID of %d" % class_to_sample)
    fid = compute_fid(gen_samples[class_to_sample], true_samples[class_to_sample])
    print("FID: %.4f" % fid)